In [1]:
import mujoco_py
import os
import numpy as np
from mujoco_dynamics import MujocoDynamics

In [2]:
xml_path = os.path.join(os.getcwd(), 'xmls', 'inverted_pendulum.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

In [9]:
sim.data.qpos[:] = [0.00184113, -0.00434254]
sim.data.qvel[:] = [0.18378366, -0.43146432]
print(sim.data.qpos, sim.data.qvel)
sim.data.ctrl[:] = [3.0]
sim.step()
print(sim.data.qpos, sim.data.qvel)

[ 0.00184113 -0.00434254] [ 0.18378366 -0.43146432]
[ 0.01102428 -0.02587148] [ 0.73359629 -1.71385134]


In [60]:
print(sim.data.qpos, sim.data.qvel)
sim.data.ctrl[:] = [1.000001]
sim.step()
print(sim.data.qpos, sim.data.qvel)

[ 0.02920693 -0.06747734] [ 0.72686904 -1.66372661]
[ 0.04553858 -0.10479942] [ 0.90616025 -2.06864007]


In [84]:
print(sim.model.opt.timestep)
print(sim.model.actuator_ctrlrange)

0.02
[[-3.  3.]]


In [107]:
print(np.concatenate([sim.data.qpos, sim.data.qvel]))
print(sim.model.nq, sim.model.nv)

[0. 0. 0. 0.]
2 2


In [5]:
dynamics = MujocoDynamics(xml_path)

In [11]:
print(dynamics.get_state())
dynamics.set_state(np.array([0.00184113, -0.00434254, 0.18378366, -0.43146432]))
print(dynamics.step(np.array([1.0])), dynamics.constrain(np.array([1.0])))

[ 0.01102428 -0.02587148  0.73359629 -1.71385134]
[ 0.00970685 -0.0227588   0.60210889 -1.40465707] [2.28478247]
